In [ ]:
!pip install plotly

In [ ]:
!pip install gradio

In [ ]:
!pip install tensorflow

In [1]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import os
import plotly.express as px
import numpy as np
from tensorflow.keras.models import load_model
import json  # Добавьте этот импорт в начало вашего скрипта
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm  # Добавьте этот импорт в начало вашего файла
from joblib import dump, load
import zipfile
import plotly.graph_objs as go
import shutil
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
class MakeMyDataSet:
    # Атрибут класса
    _input_file = "input.csv"
    _output_file = "output.csv"
    applied_methods = {}
    file_name_train = "train_data.csv"
    file_name_test =  "test_data.csv"
    file_name_json = "methods_info.json"
    file_name_seasonal = "seasonal.csv"
    file_name_trend = "trend.csv"
    scaler_filename_x = "scaler_x.joblib"
    scaler_filename_y = "scaler_y.joblib"

    #устнавливаем входной файл, с которым будем работать (расширение txt)
    def setNameInputFile(self, input_file):
        self._input_file = input_file

    def __add_prefix_to_filename(self, prefix):
        # Разделяем путь на директорию и имя файла
        directory, filename = os.path.split(self._input_file)

        # Добавляем префикс к имени файла
        new_filename = prefix + filename

        # Соединяем директорию и новое имя файла обратно в полный путь
        new_file_path = os.path.join(directory, new_filename)

        return new_file_path

    #устнавливаем выходные файлы (расширение csv)
    def setNameOutputFile(self, prefix):
        self._output_file = self.__add_prefix_to_filename(prefix)


    #преобразование числовых значений в направления ветра
    def __degrees_to_direction(self, degrees):
        if degrees < 0 or degrees > 360:
            return "Недействительный угол"
        if 337.5 <= degrees or degrees < 22.5:
            return "Север"
        elif 22.5 <= degrees < 67.5:
            return "Северо-Восток"
        elif 67.5 <= degrees < 112.5:
            return "Восток"
        elif 112.5 <= degrees < 157.5:
            return "Юго-Восток"
        elif 157.5 <= degrees < 202.5:
            return "Юг"
        elif 202.5 <= degrees < 247.5:
            return "Юго-Запад"
        elif 247.5 <= degrees < 292.5:
            return "Запад"
        elif 292.5 <= degrees < 337.5:
            return "Северо-Запад"

    def __degrees_to_sin(self, degrees):
        return np.sin(np.radians(degrees))

    def __degrees_to_cos(self, degrees):
        return np.cos(np.radians(degrees))

    #добавляем столбцы sin, cos для флюгеров
    def makeColumnsVane(self, file, columns_vane):
       try:
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

            for column in data.columns:
              if column in columns_vane:
                data[column + '_sin'] = data[column].apply(self.__degrees_to_sin)
                data[column + '_cos'] = data[column].apply(self.__degrees_to_cos)

            data.to_csv(self._output_file, index=False)
            self.applied_methods['makeColumnsVane'] = True
       except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
            return
       except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

    # Вычисление разницы времени между последовательными записями в минутах
    def __find_time_diff_min(self, data):
        # Создаем временный столбец для расчёта разницы времени
        temp_time_diff = data['date'].diff().dt.total_seconds() / 60

        # Находим наиболее частое значение разницы времени
        mode_time_diff = temp_time_diff.mode()[0]

        print(f"Наиболее частая разница во времени между записями: {mode_time_diff} минут")

        # Возвращаем результат без изменения исходного DataFrame
        return mode_time_diff

    #сохранение сезонности с учетом даты начала периода
    def __save_seasonality_with_dates(self, seasonal_component, period, filename):
        try:
            if not isinstance(seasonal_component.index, pd.DatetimeIndex):
                raise ValueError("Индекс seasonal_component должен быть временным (DatetimeIndex).")

            # Извлечение первого периода сезонности вместе с индексом дат
            first_period_seasonality = seasonal_component.head(int(period))

            # Преобразование индекса в столбец для ясности
            first_period_seasonality = first_period_seasonality.reset_index()

            # Сохранение даты и сезонной компоненты в файл CSV
            first_period_seasonality.to_csv(filename, index=False)

            print(f"Сезонная составляющая за первый период сохранена в '{filename}' с датами.")

        except ValueError as e:
            print(f"Ошибка в данных: {e}")
        except Exception as e:
            print(f"Произошла ошибка при сохранении сезонной составляющей: {e}")


    #убираем сезонность
    def deleteSeason(self, file, column_goal = ''):
        try:
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

            # Определение минимальной и максимальной даты
            min_date = data['date'].min()
            max_date = data['date'].max()

            # Вычисление разницы в днях между максимальной и минимальной датами
            date_diff = (max_date - min_date).days

            # Вычисление разницы времени между последовательными записями в минутах
            diff_time = self.__find_time_diff_min(data)
            seasonal_period = 1

            if diff_time!=0:
              if date_diff>2*365: #надо как минимум 2 периода для выявления сезонности
                #если год: 365 дней, 24 часа по 60 минут
                seasonal_period = int(365*(24*60)/diff_time)
                seasonal_period_text ='yearly'
              else:
                #если день: 24 часа по 60 минут
                seasonal_period = int((24*60)/diff_time)
                seasonal_period_text ='dayly'

            model_rez = 'additive'

            # Убедитесь, что 'date' является индексом DataFrame перед декомпозицией
            data.set_index('date', inplace=True)

            for column in data.columns:
                decomposition = sm.tsa.seasonal_decompose(data[column],
                                                          model=model_rez,
                                                          period=seasonal_period,
                                                          extrapolate_trend='freq')

                seasonal_component = decomposition.seasonal

                aligned_data, aligned_seasonal = data[column].align(decomposition.seasonal, join='left', fill_value=0)
                data[column] = aligned_data - aligned_seasonal
                data[column] = data[column] - decomposition.seasonal

             # Возвращаем 'date' в столбцы DataFrame
            data.reset_index(inplace=True)
            data.to_csv(file, index=False)

            self.__save_seasonality_with_dates(seasonal_component,
                                             seasonal_period,
                                             self.file_name_seasonal)

            self.applied_methods['deleteSeason'] = { 'extrapolate_trend': 'freq',
                                                    'model': model_rez,
                                                    'period': seasonal_period_text,
                                                    'seasonal_file': self.file_name_seasonal}

        except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
        except ValueError as e:
            print(f"Ошибка данных: {e}")
        except Exception as e:
            print(f"Произошла ошибка: {e}")


    def deleteSeasonForTest(self, file):
        try:
            if 'deleteSeason' not in self.applied_methods:
                raise ValueError("Конфигурация сезонности не найдена.")

            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

            season_config = self.applied_methods['deleteSeason']
            seasonal_file = season_config['seasonal_file']
            period = season_config['period']
            model = season_config['model']

            # Загрузка сезонности
            seasonality = pd.read_csv(seasonal_file, parse_dates=['date'])

            if period == 'yearly':
                seasonality['time_key'] = seasonality['date'].dt.dayofyear
                if 'date' in data.columns:
                    data['time_key'] = pd.to_datetime(data['date']).dt.dayofyear
                else:
                    raise ValueError("Столбец 'date' отсутствует в данных.")
            elif period == 'dayly':
                 seasonality['time_key'] = seasonality['date'].dt.hour * 60 + seasonality['date'].dt.minute
                 if 'date' in data.columns:
                    # Конвертация времени в минуты с начала дня
                    data['time_key'] = pd.to_datetime(data['date']).dt.hour * 60 + pd.to_datetime(data['date']).dt.minute
                 else:
                    raise ValueError("Столбец 'date' отсутствует в данных.")

            # Установка time_key как индекс в данных сезонности
            seasonality.set_index('time_key', inplace=True)
            data.set_index('time_key', inplace=True)

            # Переиндексация основных данных для соответствия сезонности
            aligned_seasonality = seasonality.reindex(data.index, method='nearest')
            # Слияние с использованием merge_asof
            #data = pd.merge_asof(data.sort_values('time_key'), seasonality.sort_values('time_key'), on='time_key', direction='nearest')

            # Прибавление или умножение сезонной компоненты
            #for column in data.select_dtypes(include=[np.number]):
            #    if column in seasonality.columns:
            #        if model == "additive":
            #            data[column] += data[column + '_y'].fillna(0)
            #        elif model == "multiplicative":
            #            data[column] *= data[column + '_y'].fillna(1)


            # Исключение дат из операций
            numeric_columns = data.select_dtypes(include=[np.number])

            # Прибавление или умножение сезонной компоненты к числовым данным
            for column in numeric_columns:
                if column in aligned_seasonality.columns:
                    if model == "additive":
                        data[column] += aligned_seasonality[column].fillna(0)
                    elif model == "multiplicative":
                        data[column] *= aligned_seasonality[column].fillna(1)

            # Восстановление оригинального порядка столбцов и индекса
            data.reset_index(drop=True, inplace=True)

            data.to_csv(file, index=False)

            print("Сезонная компонента успешно применена к данным.")

        except FileNotFoundError:
             print(f"Ошибка: файл '{self.file_name_seasonal}' не найден.")
        except Exception as e:
            print(f"Произошла ошибка при применении сезонности: {e}")

    #убираем тренд
    def deleteTrend(self, file):
        try:
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
            time = np.arange(len(data))

            trend_info = {}

            for column in data.columns:
                if column == 'date':
                    continue

                # Создание модели линейной регрессии и вычисление тренда
                y = data[column].values
                model = LinearRegression().fit(time.reshape(-1, 1), y)

                # Убираем тренд из данных
                trend = model.predict(time.reshape(-1, 1))
                data[column] = y - trend

                # Сохранение информации о тренде, если это требуемый столбец
                trend_info[column] = {'slope': model.coef_[0], 'intercept': model.intercept_}

            # Сохранение обработанных данных
            data.to_csv(file, index=False)

            # Сохранение информации о тренде в файл, если есть что сохранять
            if trend_info is not None:
                pd.DataFrame(trend_info).to_csv(self.file_name_trend, index=True)
                self.applied_methods['deleteTrend'] = {'trend_file': self.file_name_trend}

        except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
            return
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

    #применяем дифференциование к данным
    def applyDiffToData(self, file, is_write):
        try:
            data = pd.read_csv(file)

            # Сохраняем первые значения для числовых столбцов
            numeric_cols = data.select_dtypes(include=[np.number])
            initial_values = numeric_cols.iloc[0]

            # Применение дифференцирования только к числовым столбцам
            diff_data = numeric_cols.diff(periods=1).bfill().dropna()

            # Возвращаем нечисловые столбцы назад в DataFrame
            non_numeric_cols = data.select_dtypes(exclude=[np.number])
            diff_data = pd.concat([diff_data, non_numeric_cols], axis=1)

            # Сохранение обработанных данных
            diff_data.to_csv(file, index=False)
            print("Данные дифференцированы")

            # Сохраняем первые значения в атрибуты класса или в файл, если это необходимо
            if is_write:
                self.applied_methods['applyDiffToData'] = {'initial_values': initial_values.to_dict()}

        except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
            return
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return


    #добавляем столбцы с датой в виде года/месяца
    def transformColumnData(self, file, extract = 'year'):
        try:
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')

            min_year = data['date'].dt.year.min()
            # Добавляем столбец с годом или месяцем
            if extract == 'year':
                data['year'] = data['date'].dt.year - min_year
            elif extract == 'month':
                data['month'] = data['date'].dt.month

            # Сохранение обработанного DataFrame
            data.to_csv(self._output_file, index=False)
            self.applied_methods['transformColumnData'] = {'method': extract}

        except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
            return
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

    #агрегация данных до значений
    def agregateData(self, file, type_agregation = 'min'):
       try:
            # Загрузим данные из CSV файла.
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
            data.set_index('date', inplace=True)

            # Агрегация по 30 минутам:
            if type_agregation == '30min':
                resampled = data.resample('30min').mean()
            elif type_agregation == '1hour':
                # Агрегация по 1 часу:
                resampled = data.resample('1h').mean()
            elif type_agregation == '6hours':
                # Агрегация по 6 часам:
                resampled = data.resample('6h').mean()
            elif type_agregation == '12hours':
                # Агрегация по 12 часам:
                resampled = data.resample('12h').mean()

            self.applied_methods['agregateData'] = {'method': type_agregation}
            # Теперь запишем результаты обратно в CSV.
            resampled.to_csv(file)

       except FileNotFoundError:
           print(f"Ошибка: файл '{file}' не найден.")
           return
       except Exception as e:
           print(f"Произошла ошибка: {e}")
           return

    #нормализуем данные
    def normalizeData(self, file, column_goal='', method='minmax'):
       try:
           data = pd.read_csv(file)
           dates = data['date']

           # Выбор метода нормализации
           scaler_x = MinMaxScaler() if method == 'minmax' else StandardScaler()
           scaler_y = MinMaxScaler() if method == 'minmax' else StandardScaler()

           # Нормализация целевого столбца, если он указан
           if column_goal and column_goal in data.columns:
                # Преобразование данных целевого столбца в 2D для fit_transform
                column_data = data[[column_goal]]
                data[column_goal] = scaler_y.fit_transform(column_data)
                goal = data[column_goal]
                dump(scaler_y, self.scaler_filename_y)
                self.applied_methods['normalizeData_y'] = {'method': method,
                                                           'scaler_file': self.scaler_filename_y}

           # Нормализация всех столбцов, кроме 'date' и целевого
           columns_to_scale = data.columns.drop(['date', column_goal]) if column_goal else data.columns.drop('date')
           if columns_to_scale.empty:
                print("Warning: No columns to scale!")
           else:
                data[columns_to_scale] = scaler_x.fit_transform(data[columns_to_scale])
                dump(scaler_x, self.scaler_filename_x)
                self.applied_methods['normalizeData_x'] = {'method': method,
                                                           'scaler_file': self.scaler_filename_x}



           data['date'] = dates
           data[column_goal] = goal

           # Сохранение обработанного DataFrame
           data.to_csv(file, index=False)

       except FileNotFoundError:
            print(f"Ошибка: файл '{file}' не найден.")
            return
       except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

     #применяем нормализацию для тестовых данных
    def makeNormalizeForTest(self, file, column_goal):
        try:
           # Загрузка данных
           data = pd.read_csv(file)
           dates = data['date'].copy()


           if os.path.exists(self.scaler_filename_y):
                scaler_y = load(self.scaler_filename_y)
                column_data = data[[column_goal]]
                # Нормализация целевых столбцов
                data[column_goal] = scaler_y.transform(column_data)
                print("Данные успешно нормализованы с использованием сохранённых scaler_y.")
                goal = data[column_goal]

           # Загрузка объектов scaler
           if os.path.exists(self.scaler_filename_x):
                scaler_x = load(self.scaler_filename_x)
                # Нормализация всех столбцов, кроме 'date' и целевых столбцов
                columns_to_scale = data.columns.drop(['date', column_goal]) if column_goal else data.columns.drop('date')
                if columns_to_scale.empty:
                    print("Warning: No columns to scale!")
                else:
                    data[columns_to_scale] = scaler_x.transform(data[columns_to_scale])
                    print("Данные успешно нормализованы с использованием сохранённых scaler_x.")

           data['date'] = dates
           data[column_goal] = goal

           data.to_csv(file, index=False)

        except FileNotFoundError:
           print(f"Ошибка: файл '{file}' или scaler файлы не найдены.")
           return None
        except Exception as e:
           print(f"Произошла ошибка: {e}")
           return None

    # Функция для заполнения столбцов различными методами интерполяции
    def __fillWithInterpol(self, series, type):
            filled_series = series  # Инициализируем filled_series

            if type == 'linear':
                filled_series = series.interpolate(method='linear')
            elif type == 'spline':
                filled_series = series.interpolate(method='spline', order=3)
            elif type == 'nearest':
                filled_series = series.interpolate(method='nearest')
            elif type == 'mean':
                filled_series = series.fillna(series.mean())
            elif type == 'median':
                filled_series = series.fillna(series.median())
            else:
                filled_series = series.interpolate(method='linear')
                print(f"Тип интерполяции '{type}' не поддерживается. Поэтому был взят тип linear по умолчанию")

            return filled_series

    # Заполняем пропущенные значения в датасете
    def fillInterpol(self, file, type = 'linear'):
        try:
            data = pd.read_csv(file)
            data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
            # Установите столбец даты и времени как индекс DataFrame
            data.set_index('date', inplace=True)

            # Сортировка данных по индексу
            data = data.sort_index()

            # Генерация полной последовательности дат
            full_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='10min')

            # Расширение DataFrame для включения всех временных меток в диапазоне
            data_full = data.reindex(full_range)

            # Применение функции заполнения пропусков к объединенному DataFrame
            for column in data.columns:
              data_full[column] = self.__fillWithInterpol(data_full[column], type)

            # Сброс индекса перед сохранением в CSV
            data_full.reset_index(inplace=True)

            # Переименование столбца индекса обратно в 'date'
            data_full.rename(columns={'index': 'date'}, inplace=True)

            # Сохранение обработанного DataFrame в обратно в файл CSV
            data_full.to_csv(self._output_file, index=False)
            self.applied_methods['fillInterpol'] = {'method': type}

        except FileNotFoundError:
            print(f"Ошибка: файл '{self._output_file}' не найден.")
            return
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return


    # Делаем датасет со списоком столбцов, которые хотим выбрать
    def makeNewFileDataset(self, columns_to_select):
        try:
            # Убеждаемся, что 'date' всегда включен в список столбцов для выборки
            columns_to_select = list(set(columns_to_select + ['date']))

            # Чтение CSV файла
            data = pd.read_csv(self._input_file, usecols=columns_to_select)

            # Сохранение обработанного DataFrame в CSV
            data.to_csv(self._output_file, index=False)

        except FileNotFoundError:
            print(f"Ошибка: файл '{self._output_file}' не найден.")
            return
        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

    # Удаляем дату из данных
    def deleteDateFromDataset(self, file):
        try:
           data = pd.read_csv(file)
           if 'date' in data.columns:
               dates = data['date'].copy()
               data.drop('date', axis=1, inplace=True)  # Удаление столбца 'date' из DataFrame
               # Сохранение обработанного DataFrame в CSV
               data.to_csv(file, index=False)

        except FileNotFoundError:
            print(f"Ошибка: файл '{self._output_file}' не найден.")
            return

        except Exception as e:
            print(f"Произошла ошибка: {e}")
            return

    #сохраняем в методы название целевого столбца
    def add_goal_column_name_to_method(self, goal_radio_group):
        self.applied_methods['column_name_goal'] = goal_radio_group
    
    # Метод для сохранения словаря в JSON
    def save_methods_to_json(self, json_filename):
        with open(json_filename, 'w') as file:
            json.dump(self.applied_methods, file, indent=4)

    # Разделение датасета на тренировочную и тестовую выборки
    def splitData(self, test_size=0.2):
       try:
            data = pd.read_csv(self._output_file)

            # Определяем точку разделения
            split_point = int(len(data) * (1 - test_size))

            # Разделение данных на тренировочную и тестовую выборки
            train_data = data[:split_point]
            test_data = data[split_point:]

            # Сохранение выборок в разные файлы
            train_data.to_csv(self.file_name_train, index=False)
            test_data.to_csv(self.file_name_test, index=False)
            #test_data.to_csv("test_for_test.csv", index=False)

       except FileNotFoundError:
            print(f"Ошибка: файл '{self._input_file}' не найден.")
            return
       except Exception as e:
            print(f"Произошла ошибка: {e}")
            return





In [3]:
#класс для применения методов к данным
class ApplyMethodsToData:
    def __init__(self, initial_data, file_path):
        self.data = initial_data.copy()
        self.file_path = file_path
        if 'date' in self.data.columns:
            self.data['date'] = pd.to_datetime(self.data['date'], format='%Y-%m-%d %H:%M:%S')
            #self.data.set_index('date', inplace=True)
        self.methods = {}
        self.sampling_window_size = None
        self.test_data_offset = None
        self.column_name_goal = None  

    #Загрузка методов обработки из JSON файла.
    def load_methods_from_json(self, json_filename):
        try:
            with open(json_filename, 'r') as file:
                self.methods = json.load(file)
                # Чтение дополнительных параметров
            self.sampling_window_size = self.methods.get("sampling_window_size", None)
            self.test_data_offset = self.methods.get("test_data_offset", None)
            self.column_name_goal = self.methods.get("column_name_goal", None)
            print(f"Loaded sampling_window_size: {self.sampling_window_size}")
            print(f"Loaded test_data_offset: {self.test_data_offset}")
            print(f"Loaded column_name_goalt: {self.column_name_goal}")
        except FileNotFoundError:
            print(f"Ошибка: файл '{json_filename}' не найден.")
        except json.JSONDecodeError:
            print(f"Ошибка: файл '{json_filename}' содержит некорректный JSON.")

    #Обработка методов в обратном порядке в соответствии с загруженным JSON."""
    def process_methods(self, test = 0):
        #for predict
        methods_dict = {
                            "normalizeData_y": self.applyNormalizeToData,
                            "deleteTrend": self.applyTrendToData,
                            "deleteSeason": self.applySeasonToData,
                            "applyDiffToData": self.applyUnDiffToData
                           }

        # Обработка методов в обратном порядке
        for method, details in reversed(list(self.methods.items())):
            if method in methods_dict:
                if isinstance(details, dict):  # Проверяем, являются ли детали словарём
                    print(f"Применение {method} с параметрами {details}...")
                    methods_dict[method](**details)  # Распаковка словаря в аргументы функции
                else:
                    print(f"Применение {method}...")
                    methods_dict[method]()  # Вызов метода без параметров


    #применить нормализацию к данным
    def applyNormalizeToData(self, scaler_file, method):
       try:

            # Загрузка объекта scaler
            scaler = load(os.path.join(self.file_path, scaler_file))

            # Сохранение столбца с датами, если он есть
            is_data = 0
            if 'date' in self.data.columns:
                is_data = 1
                dates = self.data['date'].copy()

            # Определение столбцов для нормализации
            columns_to_scale = self.data.columns.drop('date', errors='ignore')  # Игнорируем ошибки, если 'date' нет
            print(columns_to_scale)
            # Применение нормализации
            temp = self.data[columns_to_scale]
            temp = scaler.inverse_transform(temp)

            self.data[columns_to_scale] = temp
            # Возвращение столбца с датами обратно в DataFrame, если он был сохранён
            if is_data:
                self.data['date'] = dates
            print("Данные успешно восстановлены из нормализованной формы.")

       except FileNotFoundError:
            print(f"Ошибка: файл '{scaler_file}' не найден.")
       except Exception as e:
            print(f"Произошла ошибка: {e}")


    #применить обратную дифферециацию к данным
    def applyUnDiffToData(self, initial_values):
        try:
            # Создаем копию DataFrame, чтобы избежать изменения исходных данных
            numeric_data = self.data.copy()

            # Удаляем столбец с датами из данных, предназначенных для восстановления
            pr_restore = 0
            if 'date' in numeric_data.columns:
                pr_restore = 1
                dates = numeric_data['date']
                numeric_data.drop('date', axis=1, inplace=True)

            # Восстановление числовых данных из дифференцированных значений
            restored_data = numeric_data.cumsum() + initial_values

            # Возвращаем столбец с датами обратно в DataFrame
            if pr_restore:
                restored_data['date'] = dates

            # Поскольку cumsum() не включает первую строку начальных значений, заменяем первую строку на начальные значения
            for col in restored_data.columns:
                if col != 'date':
                    restored_data[col].iloc[0] = initial_values[col]

            # Сохранение восстановленных данных
            self.data = restored_data
            print("Данные успешно восстановлены из дифференцированной формы.")

        except Exception as e:
                print(f"Произошла ошибка: {e}")

    #применить тренд к данным
    def applyTrendToData(self, trend_file):
        try:
            # Загрузка информации о тренде
            trend_info = pd.read_csv(os.path.join(self.file_path,trend_file), index_col=0)
            time = np.arange(len(self.data))

            # Применение тренда ко всем столбцам, для которых он был сохранен
            for column in trend_info.columns:
                if column in self.data.columns:
                    # Извлечение параметров тренда
                    slope = trend_info.at['slope', column]
                    intercept = trend_info.at['intercept', column]

                    # Вычисление тренда
                    trend = slope * time + intercept

                    # Добавление тренда к данным
                    self.data[column] = self.data[column] + trend
                    print(f"Тренд успешно применён к столбцу {column}.")

        except FileNotFoundError:
                print(f"Ошибка: файл '{trend_file}' не найден.")
        except Exception as e:
                print(f"Произошла ошибка: {e}")

    #применить сезонность к данным
    def applySeasonToData(self, period, seasonal_file, model, extrapolate_trend):
        try:
            # Загрузка сезонности
            seasonality = pd.read_csv(os.path.join(self.file_path,seasonal_file), parse_dates=['date'])
            if period == 'yearly':
                seasonality['time_key'] = seasonality['date'].dt.dayofyear
                if 'date' in self.data.columns:
                    self.data['time_key'] = pd.to_datetime(self.data['date']).dt.dayofyear
                else:
                    raise ValueError("Столбец 'date' отсутствует в данных.")
            elif period == 'dayly':
                 seasonality['time_key'] = seasonality['date'].dt.hour * 60 + seasonality['date'].dt.minute
                 if 'date' in self.data.columns:
                    # Конвертация времени в минуты с начала дня
                    self.data['time_key'] = pd.to_datetime(self.data['date']).dt.hour * 60 + pd.to_datetime(self.data['date']).dt.minute
                 else:
                    raise ValueError("Столбец 'date' отсутствует в данных.")

            # Установка time_key как индекс в данных сезонности
            seasonality.set_index('time_key', inplace=True)
            self.data.set_index('time_key', inplace=True)

            # Переиндексация основных данных для соответствия сезонности
            aligned_seasonality = seasonality.reindex(self.data.index, method='nearest')

            # Исключение дат из операций
            numeric_columns = self.data.select_dtypes(include=[np.number])

            # Прибавление или умножение сезонной компоненты к числовым данным
            for column in numeric_columns:
                if column in aligned_seasonality.columns:
                    if model == "additive":
                        self.data[column] += aligned_seasonality[column].fillna(0)
                    elif model == "multiplicative":
                        self.data[column] *= aligned_seasonality[column].fillna(1)

            # Восстановление оригинального порядка столбцов и индекса
            self.data.reset_index(drop=True, inplace=True)

            print("Сезонная компонента успешно применена к данным.")

        except FileNotFoundError:
             print(f"Ошибка: файл '{seasonal_file}' не найден.")
        except Exception as e:
            print(f"Произошла ошибка при применении сезонности: {e}")

    def saveToCSV(self, file_name_test):
        if 'date' in self.data.columns:
               dates = self.data['date'].copy()
               self.data.drop('date', axis=1, inplace=True)  # Удаление столбца 'date' из DataFrame

        self.data.to_csv(file_name_test, index=False)


In [4]:
# Стили CSS для настройки фона и текста
css = """
.gradio-container {
    background: none !important;
    background-image: url('/content/vetroyenergetika.jpg') !important;
    background-position: center;
    background-repeat: no-repeat;
    background-size: cover;
    color: #4CAF50;
    background-color: black !important;
}

h3, h2{
   color: #4CAF50; /* Зеленый цвет текста */
   font-family: Arial, sans-serif;
}

.button_my {
    background: none !important;
    background-color: #4CAF50 !important;
    color: black;
    padding: 10px 20px;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    font-size: 16px;
}

.button_my:hover {
    background-color: #70bf73 !important;
}

.tab_my {
    background: none !important;
    background-image: url('/content/images.jpeg') !important;
    border: 1px solid #ddd !important; /* Граница вкладок */
    padding: 10px !important; /* Отступы внутри вкладок */
}

.small-file-input {
    width: 150px; /* Устанавливаем ширину */
    height: 80px !important; /* Устанавливаем высоту */
}

upload_instructions {
    font-size: 16px; /* Изменяем размер шрифта */
}

"""

In [5]:
# Глобальные переменные для хранения данных и модели
global_data = None

# Словари модлей для разных вышек

model_choose_telichki = {
    'CNN': 'array_model/model_cnn_telichki.zip',
    'RNN': 'array_model/model_rnn_telichki.zip',
    'LSTM': 'array_model/model_lstm_telichki.zip',
    'GRU': 'array_model/model_gru_telichki.zip'
}

model_choose_ust_har = {
    'CNN': 'array_model/model_cnn_ust_harizovo.zip',
    'RNN': 'array_model/model_rnn_ust_harizovo.zip',
    'LSTM': 'array_model/model_lstm_ust_harizovo.zip',
    'GRU': 'array_model/model_gru_ust_harizovo.zip'
}

model_choose_ossora = {
    'CNN': 'array_model/model_cnn_ossora.zip',
    'RNN': 'array_model/model_rnn_ossora.zip',
    'LSTM': 'array_model/model_lstm_ossora.zip',
    'GRU': 'array_model/model_gru_ossora.zip'
}

model_choose_manila = {
    'CNN': 'array_model/model_cnn_manila.zip',
    'RNN': 'array_model/model_rnn_manila.zip',
    'LSTM': 'array_model/model_lstm_manila.zip',
    'GRU': 'array_model/model_gru_manila.zip'
}

model_choose_palana = {
    'CNN': 'array_model/model_cnn_palana.zip',
    'RNN': 'array_model/model_rnn_palana.zip',
    'LSTM': 'array_model/model_lstm_palana.zip',
    'GRU': 'array_model/model_gru_palana.zip'
}

model_choose_ust_kam = {
    'CNN': 'array_model/model_cnn_ust_kam.zip',
    'RNN': 'array_model/model_rnn_ust_kam.zip',
    'LSTM': 'array_model/model_lstm_ust_kam.zip',
    'GRU': 'array_model/model_gru_ust_kam.zip'
}


# Загрузка моделей и создание словаря
models = {
    'Телички': model_choose_telichki,
    'Усть-Харизово': model_choose_ust_har,
    'Оссора': model_choose_ossora,
    'Манила': model_choose_manila,
    'Палана': model_choose_palana,
    'Усть-Камчатск': model_choose_ust_kam
}


In [6]:
# Словарь методов заполнения пропусков
methods = {
    'Линейная интерполяция': 'linear',
    'Сплайн-интерполяция':'spline',
    'Интерполяция ближайшими значениями':'nearest',
    'Заполнение средним значением ':'mean',
    'Заполнение медианным значением ':'median'
}

In [7]:
# Словарь методов scaler
normalize = {
    'MinMaxScaler': 'minmax',
    'StandartScaler':'standart'
}

In [8]:
# Словарь методов трансформации даты
transform_data_methods = {
    'Номер месяца': 'month',
    'Номер года':'year'
}


In [9]:
# Словарь методов трансформации даты
agregation_list = {
    '30 минут': '30min',
    '1 час':'1hour',
    '6 часов':'6hours',
    '12 часов':'12hours'
}

In [10]:
#Папка, куда будут разархивироваться данные
destination_folder = "temp"

In [11]:
#находим json файл в папке
def find_first_json_file(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                return os.path.join(root, file)  # Возвращает путь к первому найденному файлу JSON и останавливает поиск
    return None  # Если JSON файл не найден, вернуть None

In [12]:
#находим модель h5, keras файл в папке
def find_first_model_file(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.h5') or file.endswith('.keras'):
                return os.path.join(root, file)  # Возвращает путь к первой найденной модели и останавливает поиск
    return None  # Если файл модели не найден, вернуть None

In [13]:
#загружаем необходимые данные для прогнозирования
def load_dataset(tower, model_name, file):

    global global_data
    global json_file_path
    global sampling_window_size
    global test_data_offset
    global column_name_goal

    #проверяем выбрана ли башня
    if tower not in models:
        return "Башня не выбрана"

    # проверяем модель для башни, берем ссылку на архив
    if model_name in models[tower]:
        zip_path = models[tower][model_name]
    else:
        return "Данные для модели не загружены"

    # Удаление папки, если она существует
    if os.path.exists(destination_folder):
        shutil.rmtree(destination_folder)
    
    # Создание папки
    os.makedirs(destination_folder)
 
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
      zip_ref.extractall(destination_folder)

    # Найти первый JSON файл в этой папке
    json_file_path = find_first_json_file(destination_folder)
    if json_file_path:
        print(f"Найден JSON файл: {json_file_path}")
    else:
        return "JSON файл не найден."

    # Загрузка тестового файла
    if file.name.endswith('.csv'):
        # Обработка CSV файла
        data = pd.read_csv(file)
    else:
        return "Неподдерживаемый формат файла"

    global_data = data.copy()  # Сохраните загруженные данные в глобальной переменной
    global_data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

    #применяем методы в обратном порядке
    processor = ApplyMethodsToData(data, destination_folder)
    processor.load_methods_from_json(json_file_path)
    processor.process_methods()
    sampling_window_size = processor.sampling_window_size
    test_data_offset = processor.test_data_offset
    column_name_goal = processor.column_name_goal
    
    data_processor = processor.data

    # Создание графика
    if 'date' in data.columns:
         data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
         data.set_index('date', inplace=True)

    # Установка столбца с датой как индекса DataFrame
    if 'date' in data_processor.columns:
        data_processor['date'] = pd.to_datetime(data_processor['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
        data_processor.set_index('date', inplace=True)
    else:
        return "Столбец с датой не найден"

    #data_test = pd.read_csv("test_for_test.csv")
    #if 'date' in data_test.columns:
    #    data_test['date'] = pd.to_datetime(data_test['date'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    #    data_test.set_index('date', inplace=True)

    fig = go.Figure()

    # Добавление исходных данных на график
    for column in data.columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[column],
                                 mode='lines', name=f'Original - {column}'))


    # Добавление обработанных данных на график
    for column in data_processor.columns:
        fig.add_trace(go.Scatter(x=data_processor.index, y=data_processor[column],
                                 mode='lines', name=f'Processed - {column}'))


    # Добавление обработанных данных на график
    #for column in data_test.columns:
    #    fig.add_trace(go.Scatter(x=data_test.index, y=data_test[column],
    #                             mode='lines', name=f'Test - {column}'))

    fig.update_layout(title='Сравнение исходных и обработанных данных',
                      xaxis_title='Дата',
                      yaxis_title='Значения',
                      legend_title='Легенда')


    return fig

In [14]:
# Функция для создания окон данных
def create_dataset(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)


In [35]:
#делаем предсказание модели
def make_prediction(duration):
    global global_data
    global json_file_path
    global sampling_window_size
    global test_data_offset
    global column_name_goal

    message = ''

    # Найти первый файл модели в этой папке
    model_file_path = find_first_model_file(destination_folder)
    if model_file_path:
        print(f"Найден файл модели: {model_file_path}")
        model = load_model(model_file_path)        
    else:
        print("Файл модели не найден.")
        model = None

    if model is None or global_data is None:
        return "Модель или данные не загружены"

    # Создаем временный столбец для расчёта разницы времени
    temp_time_diff = global_data['date'].diff().dt.total_seconds() / 60

    # Находим наиболее частое значение разницы времени в минутах
    mode_time_diff = temp_time_diff.mode()[0]

    #узнаем, сколько в одном дню должно быть подсчетов
    N = int(24*60/mode_time_diff);
    # предсказание на N шагов
    input_data = global_data.iloc[test_data_offset:duration*N]

    if 'date' in global_data.columns:
        dates = input_data['date'].copy()  # Создаем копию столбца с датами
        input_data = input_data.drop('date', axis=1)  # Удаляем столбец с датами из основного DataFrame

    # Подготовка данных
    X, _ = create_dataset(input_data[column_name_goal].values, sampling_window_size)

    # Решейпинг X 
    X = X.reshape((X.shape[0], sampling_window_size, 1))

    # Прогнозирование
    prediction_df = model.predict(X)
    prediction_df = pd.DataFrame(prediction_df, columns=[column_name_goal])
        
    # Получение предсказаний из DataFrame
    predictions = prediction_df[column_name_goal].values 
    input_data = input_data.iloc[sampling_window_size:]
    y_true = input_data[column_name_goal].values
 
    print(len(y_true), len(predictions))
    
    # Расчет MAE
    mae = mean_absolute_error(y_true, predictions)
    print('Средняя абсолютная ошибка (MAE):', mae)
    
    # Расчет MSE
    mse = mean_squared_error(y_true, predictions)
    print('Среднеквадратичная ошибка (MSE):', mse)
    
    #применяем методы в обратном порядке
    processor = ApplyMethodsToData(prediction_df, destination_folder)
    processor.load_methods_from_json(json_file_path)
    processor.process_methods()
    data_processor = processor.data

    processor_test = ApplyMethodsToData(input_data, destination_folder)
    processor_test.load_methods_from_json(json_file_path)
    processor_test.process_methods()
    data_processor_test = processor_test.data

    dates = dates[sampling_window_size:].reset_index(drop=True) 
    # Предполагаем, что `dates` имеет нужную длину. Если `prediction_df` был изменен, убедитесь, что даты соответствуют.
    prediction_df['date'] = dates # Сбрасываем индекс на случай, если это нужно
    prediction_df.set_index('date', inplace=True)
    
    # Аналогично для data_processor, если у вас есть обработанные данные
    data_processor['date'] = dates
    data_processor.set_index('date', inplace=True)

    data_processor_test = data_processor_test.reset_index(drop=True)
    data_processor_test['date'] = dates  # Сбрасываем индекс на случай, если это нужно
    data_processor_test.set_index('date', inplace=True)
    fig = go.Figure()

    # Добавление не обработанных предсказанных данных на график
    for column in prediction_df.columns:
        fig.add_trace(go.Scatter(x=prediction_df.index, y=prediction_df[column],
                                 mode='lines', name=f'Predict - {column}'))


    # Добавление обработанных предсказанных данных на график
    for column in data_processor.columns:
        fig.add_trace(go.Scatter(x=data_processor.index, y=data_processor[column],
                                 mode='lines', name=f'Predict recovery - {column}'))

    # Добавление истинных данных на график
    for column in data_processor_test.columns:
        fig.add_trace(go.Scatter(x=data_processor_test.index, y=data_processor_test[column],
                                 mode='lines', name=f'Test - {column}'))


    fig.update_layout(title='Сравнение исходных и обработанных данных',
                      xaxis_title='Дата',
                      yaxis_title='Значения',
                      legend_title='Легенда')


    return fig, mae, mse


In [16]:
def update_models_dropdown(tower):
    if tower in models:
        # Получаем список моделей для выбранной вышки
        model_names = models.get(tower, [])
        return gr.Dropdown(label="", choices=list(model_names.keys()))

In [17]:
def show_methods(checkbox_value):
    if checkbox_value:
        return gr.Dropdown(label="Выберите метод", choices=list(methods.keys()), visible=True)
    else:
        # Скрыть выпадающий список, если чекбокс не отмечен
        return gr.Dropdown(label="Выберите метод", choices=list(methods.keys()), visible=False)


In [18]:
def show_normalize(checkbox_value):
    if checkbox_value:
        return gr.Dropdown(label="Выберите метод", choices=list(normalize.keys()), visible=True)
    else:
        # Скрыть выпадающий список, если чекбокс не отмечен
        return gr.Dropdown(label="Выберите метод", choices=list(normalize.keys()), visible=False)


In [19]:
def show_agregation(checkbox_value):
    if checkbox_value:
        return gr.Dropdown(label="Выберите период", choices=list(agregation_list.keys()), visible=True)
    else:
        # Скрыть выпадающий список, если чекбокс не отмечен
        return gr.Dropdown(label="Выберите период", choices=list(agregation_list.keys()), visible=False)


In [20]:
def read_csv(file):
    if file is None:
        return gr.CheckboxGroup(choices=[], label="Выберите столбцы")
    try:
        df = pd.read_csv(file)

        # Определяем, есть ли столбец, который можно интерпретировать как дату
        for column in df.columns:
            # Попытка преобразования столбца в datetime, если не удается, игнорируем ошибки
            try:
                df[column] = pd.to_datetime(df[column])
                df.rename(columns={column: 'date'}, inplace=True)
                df.to_csv(file, index=False)
                break  # Прекращаем цикл после первого успешного преобразования
            except:
                continue
        # Исключаем столбец 'date' из списка столбцов
        columns = [col for col in df.columns if col != 'date']


        header_checkboxes.value = None
        goal_radio_group.value = None

        #clear_checkbox_group()
        clear_goal_radio_group()

        return gr.CheckboxGroup(choices=columns, label="Выберите столбцы")
    except Exception as e:
        return []

In [21]:
def update_fluger_checkboxes(add_fluger, selected_headers):
    if add_fluger and selected_headers:
        # Возвращаем список словарей, каждый из которых представляет чекбокс
        return gr.CheckboxGroup(choices=selected_headers, label="Выберите столбцы для флюгера", visible=True)
    return gr.CheckboxGroup(choices=[], label="", visible=False)

In [22]:
def show_radio_buttons(is_checked):
    # Если чекбокс активирован, показываем радиокнопки
    if is_checked:
        return gr.Radio(choices=list(transform_data_methods.keys()), label="Выберите тип", visible=True)
    # В противном случае скрываем радиокнопки
    return gr.Radio(choices=[], visible=False)

In [23]:
def update_radio_options(checkbox_selection):
    # Возвращает список выборов, основанный на выборе в CheckboxGroup
        return gr.Radio(choices=checkbox_selection, label="Выберите колонку", visible=True)


In [24]:
def clear_goal_radio_group():
    return gr.Radio(choices=[],label="Выберите колонку")


In [25]:
def clear_checkbox_group():
    return gr.CheckboxGroup(choices=[], label="Выберите столбцы")

In [26]:
def create_zip_archive(archive_path, dataset_processor, normalize_set):
    with zipfile.ZipFile(archive_path, 'w') as zipf:
        # Список файлов для архивации
        files_to_zip = [
            dataset_processor.file_name_train,
            dataset_processor.file_name_test,
            dataset_processor.file_name_json
        ]

        # Если используется нормализация, добавляем файл нормализатора
        if normalize_set:
            files_to_zip.append(dataset_processor.scaler_filename_y)
            files_to_zip.append(dataset_processor.scaler_filename_x)

        # Перебираем файлы и добавляем их в архив, если они существуют
        for file_path in files_to_zip:
            if os.path.exists(file_path):  # Проверка на существование файла
                zipf.write(file_path, os.path.basename(file_path))
            else:
                print(f"Файл {file_path} не найден и не будет добавлен в архив.")


In [27]:
def process_dataset(file,
                    columns,
                    fill_gaps,
                    fill_method,
                    remove_seasonality,
                    remove_trend,
                    fluger,
                    fluger_columns,
                    transform_date,
                    date_transform_method,
                    agregate_data,
                    agregate_method,
                    normalize_set,
                    normalize_method,
                    goal_radio_group,
                    split_percentage):
    # Создаем объект класса
    dataset_processor = MakeMyDataSet()
    # Обнуляем applied_methods
    dataset_processor.applied_methods = {}

    # Устанавливаем входной и выходной файлы
    dataset_processor.setNameInputFile(file.name)
    prefix = "rez_"
    dataset_processor.setNameOutputFile(prefix)

    # Выбор столбцов и создание нового датасета
    dataset_processor.makeNewFileDataset(columns)

    # Применяем выбранные методы

    #заполняем пропуски
    if fill_gaps:
        # Преобразование метки метода в его ключ
        fill_method_key = methods.get(fill_method, None)
        dataset_processor.fillInterpol(dataset_processor._output_file, fill_method_key)

    #добавить столбцы к флюгеру
    if fluger:
        dataset_processor.makeColumnsVane(dataset_processor._output_file, fluger_columns)

    #добавить столбцы к дате
    if transform_date:
        date_transform_method_key = transform_data_methods.get(date_transform_method, None)
        dataset_processor.transformColumnData(dataset_processor._output_file, date_transform_method_key)

    #агрегация данных
    if agregate_data:
        agregate_key = agregation_list.get(agregate_method, None)
        dataset_processor.agregateData(dataset_processor._output_file, agregate_key)

    # Разделение датасета на тренировочную и тестовую выборки
    dataset_processor.splitData(test_size=(100 - split_percentage) / 100)

    #убрать сезонность
    if remove_seasonality:
        dataset_processor.deleteSeason(dataset_processor.file_name_train)
        dataset_processor.deleteSeasonForTest(dataset_processor.file_name_test)

    #убрать тренд
    if remove_trend:
        #dataset_processor.deleteTrend(dataset_processor.file_name_train)
        #dataset_processor.deleteTrend(dataset_processor.file_name_test)
        dataset_processor.applyDiffToData(dataset_processor.file_name_train, is_write = 0)
        dataset_processor.applyDiffToData(dataset_processor.file_name_test, is_write =1)


    if normalize_set:
       # Преобразование метки метода в его ключ
       normalize_method_key = normalize.get(normalize_method, None)
       dataset_processor.normalizeData(dataset_processor.file_name_train,
                                       goal_radio_group,
                                       normalize_method_key)
       dataset_processor.makeNormalizeForTest(dataset_processor.file_name_test,
                                              goal_radio_group)

    dataset_processor.add_goal_column_name_to_method(goal_radio_group)
  
    dataset_processor.save_methods_to_json(dataset_processor.file_name_json)
    # Удаляем дату из данных
    dataset_processor.deleteDateFromDataset(dataset_processor.file_name_train)
    #dataset_processor.deleteDateFromDataset(dataset_processor.file_name_test)

    # Создаем архив с файлами для скачивания
    archive_path = "files_to_download.zip"
    create_zip_archive(archive_path, dataset_processor, normalize_set)
    return archive_path


In [ ]:
# Элементы интерфейса
with gr.Blocks(css=css) as demo:
  with gr.Tabs():
################################################################################1-я вкладка
    with gr.Tab(label = "Прогноз"):
      with gr.Row():
            with gr.Column():
              with gr.Row():
                  gr.Markdown("### Выбрать вышку")
              with gr.Row():
                  dropdown_towers = gr.Dropdown(label="", choices=list(models.keys()))

            with gr.Column():
              with gr.Row():
                  gr.Markdown("### Выбрать модель")
              with gr.Row():
                  dropdown_models = gr.Dropdown(label=" ", choices=[])

            with gr.Column():
              with gr.Row():
                gr.Markdown("### Выбрать файл данных")
              with gr.Row():
                file_input = gr.File(label="Загрузить файл", elem_classes="small-file-input")
              with gr.Row():
                load_button = gr.Button("Загрузить данные в систему", elem_classes="button_my")

      with gr.Row():
              gr.Markdown("## Входные данные")
      with gr.Row():
              load_button.click(fn=load_dataset,
                                inputs=[dropdown_towers, dropdown_models, file_input],
                                outputs=gr.Plot(label="Результат загрузки"))
      with gr.Row():
              gr.Markdown("## Настройки прогноза")
      with gr.Row():
          with gr.Column():
              duration_slider = gr.Slider(minimum=1, maximum=365, label="Длительность прогноза, день")
          with gr.Column():
              predict_button = gr.Button("Начать прогноз", elem_classes="button_my")
      with gr.Row():
              gr.Markdown("## Предсказание")
      with gr.Row():
              output_plot = gr.Plot(label="График прогноза")

      # Размещение текстовых полей на отдельной строке
      with gr.Row():
        with gr.Column():
            with gr.Row():
                gr.Markdown("### MAE")
            with gr.Row():
                output_mae = gr.Textbox(label="")
        with gr.Column():
            with gr.Row():
                gr.Markdown("### MSE")
            with gr.Row():
                output_mse = gr.Textbox(label="")
        #with gr.Column():
        #    with gr.Row():
        #        gr.Markdown("### MAPE")
        #    with gr.Row():
        #        output_mape = gr.Textbox(label="")


    # Настройка действия для кнопки
    predict_button.click(fn=make_prediction,
                         inputs=[duration_slider],
                         outputs=[output_plot, output_mae, output_mse])

     # Обработчик событий изменения выбора вышки
    dropdown_towers.change(fn=update_models_dropdown, inputs=dropdown_towers, outputs=dropdown_models)
################################################################################2-я вкладка
    with gr.Tab(label = "Подготовка датасета", elem_classes="tab_my"):
      with gr.Row():
          with gr.Column():
              with gr.Row():
                gr.Markdown("### Выбрать файл данных")
              with gr.Row():
                file_input2 = gr.File(label="Загрузить файл", elem_classes="small-file-input")
              with gr.Row():
                load_button2 = gr.Button("Загрузить", elem_classes="button_my")
              # Динамическое обновление для чекбоксов
              with gr.Row():
                  # Пустой контейнер для чекбоксов
                  header_checkboxes = gr.CheckboxGroup(choices=[], label="Выберите столбцы", interactive = True)  # Группа для динамически создаваемых чекбоксов

          with gr.Column():
                with gr.Row():
                     gr.Markdown("### Выбрать методы подготовки датасета")

                with gr.Row():
                  with gr.Column():
                     fill_gaps_checkbox = gr.Checkbox(label="Заполнить пропуски")
                  with gr.Column():
                     methods_dropdown = gr.Dropdown(label="Выберите метод", choices=[], visible=False, interactive = True)

                with gr.Row():
                    remove_seasonality_checkbox = gr.Checkbox(label="Убрать сезонность")

                with gr.Row():
                    remove_trend_checkbox = gr.Checkbox(label="Убрать тренд")

                with gr.Row():
                  with gr.Column():
                    fluger_checkbox = gr.Checkbox(label="Добавить столбцы для флюгера")
                  with gr.Column():
                    fluger_checkboxes_group = gr.CheckboxGroup(choices=[], label="Выберите столбцы для флюгера", visible=False, interactive=True)

                with gr.Row():
                    with gr.Column():
                        transform_data_checkbox = gr.Checkbox(label="Трансформировать дату")
                    with gr.Column():
                        # Контейнер для радиокнопок
                        radio_buttons = gr.Radio(choices=[], visible=False, interactive = True)

                with gr.Row():
                    with gr.Column():
                        agregation_checkbox = gr.Checkbox(label="Агрегировать данные")
                    with gr.Column():
                        agregation_dropdown = gr.Dropdown(label="Выберите период", choices=[], visible=False, interactive = True)

                with gr.Row():
                    with gr.Column():
                        normalize_checkbox = gr.Checkbox(label="Нормализовать данные")
                    with gr.Column():
                        normalize_dropdown = gr.Dropdown(label="Выберите scaler", choices=[], visible=False, interactive = True)


                with gr.Row():
                        gr.Markdown("### Выбрать целевой ряд")
                        goal_radio_group = gr.Radio(label=" ", visible=True, interactive = True)

                with gr.Row():
                    dataset_slider = gr.Slider(minimum=1, maximum=100, label="Разделение датасета на train/test, %", interactive = True)
                with gr.Row():
                    make_dataset = gr.Button("Создать датасет", elem_classes="button_my")
                with gr.Row():
                    download_button = gr.File(label="Скачать датасет", interactive=False, elem_classes="small-file-input")


     # При изменении состояния чекбокса вызывается функция update_dropdown_visibility
    fill_gaps_checkbox.change(fn=show_methods, inputs=fill_gaps_checkbox, outputs=methods_dropdown)
    normalize_checkbox.change(fn=show_normalize, inputs=normalize_checkbox, outputs=normalize_dropdown)

    # Обработчик для кнопки "Загрузить"
    load_button2.click(fn=clear_goal_radio_group, outputs=goal_radio_group)
    load_button2.click(fn=clear_checkbox_group, outputs=header_checkboxes)

    load_button2.click(fn=read_csv, inputs=file_input2, outputs=header_checkboxes)

    agregation_checkbox.change(fn=show_agregation, inputs=agregation_checkbox, outputs=agregation_dropdown)
    # Связываем чекбокс "Добавить столбцы для флюгера" с функцией update_fluger_checkboxes
    fluger_checkbox.change(fn=update_fluger_checkboxes, inputs=[fluger_checkbox, header_checkboxes], outputs=fluger_checkboxes_group)
    # Обновление радиокнопок на основе выбора в CheckboxGroup
    header_checkboxes.change(fn=update_radio_options, inputs=header_checkboxes, outputs=goal_radio_group)


    transform_data_checkbox.change(fn=show_radio_buttons, inputs=transform_data_checkbox, outputs=radio_buttons)
    make_dataset.click(fn=process_dataset,
                       inputs=[file_input2,
                               header_checkboxes,
                               fill_gaps_checkbox,
                               methods_dropdown,
                               remove_seasonality_checkbox,
                               remove_trend_checkbox,
                               fluger_checkbox,
                               fluger_checkboxes_group,
                               transform_data_checkbox,
                               radio_buttons,
                               agregation_checkbox,
                               agregation_dropdown,
                               normalize_checkbox,
                               normalize_dropdown,
                               goal_radio_group,
                               dataset_slider],
                       outputs=download_button)
# Запуск интерфейса
demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------
Найден JSON файл: temp\methods_info.json
Loaded sampling_window_size: 10
Loaded test_data_offset: 10
Loaded column_name_goalt: Anem_40S_10_avg
Применение normalizeData_y с параметрами {'method': 'minmax', 'scaler_file': 'scaler_y.save'}...
Index(['Anem_40S_10_avg'], dtype='object')
Данные успешно восстановлены из нормализованной формы.
Найден файл модели: temp\my_model_cnn.keras
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
2860 2860
Средняя абсолютная ошибка (MAE): 0.024337350552486894
Среднеквадратичная ошибка (MSE): 0.0011565265645214777
Loaded sampling_window_size: 10
Loaded test_data_offset: 10
Loaded column_name_goalt: Anem_40S_10_avg
Применение normalizeData_y с параметрами {'method': 'minmax', 'scaler_file': 'scaler_y.save'}...
Index(['Anem_40S_10_avg'], dtype='object')
Данные успешно восстановлены из нормализованной формы.
Loaded sampling_window_size: 10
Loaded test_data_offs

In [89]:
json_file_path = 'd:/marina/TEST_WIND_ust_kam/methods_info.json'

# Чтение JSON файла
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Модификация данных JSON
data['sampling_window_size'] = 10  # Пример размера окна выборки
data['test_data_offset'] = 10       # Пример размера окна отступа в тестовых данных

# Сохранение изменений в JSON файл
with open(json_file_path, 'w') as file:
    json.dump(data, file, indent=4)  # Использование indent для улучшения читаемости

print("JSON файл успешно обновлен.")

JSON файл успешно обновлен.
